<a href="https://colab.research.google.com/github/Fguarien/Analogique/blob/master/cleanup_urls_lists.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import files
uploaded = files.upload()

Saving whiteliste-urls.csv to whiteliste-urls.csv


In [15]:
import pandas as pd

# Charger le fichier Excel
file_name = 'whiteliste-urls.csv'  # Utilisez le nom du fichier que vous avez téléchargé
url_data = pd.read_csv(file_name)

# Afficher les premières lignes pour vérifier
url_data.head()



,URL
0,addon-stib-mivb.eu.goskope.com
1,gateway-stib-mivb.eu.goskope.com
2,gateway-backup-stib-mivb.eu.goskope.com
3,download-stib-mivb.eu.goskope.com
4,achecker-stib-mivb.eu.goskope.com


In [21]:
import pandas as pd
import fnmatch

# Charger les données
url_data = pd.read_csv('whiteliste-urls.csv')

# Liste des wildcards à vérifier
wildcards = ['*.example.com', '*.infomaniak.com', '*.jsdelivr.net']  # Ajoutez vos wildcards ici

# Fonction pour vérifier si une URL est couverte par un wildcard
def is_covered_by_wildcard(url, wildcards):
    for wildcard in wildcards:
        # Remplacez le wildcard par un pattern compatible avec fnmatch
        pattern = wildcard.replace('*.', '*')
        if fnmatch.fnmatch(url, pattern):
            return True
    return False

# Ajouter une colonne pour indiquer si l'URL est couverte par un wildcard
url_data['Covered by Wildcard'] = url_data['URL'].apply(lambda url: is_covered_by_wildcard(url, wildcards))

# Séparer les URLs couvertes et non couvertes
covered_urls = url_data[url_data['Covered by Wildcard']]
non_covered_urls = url_data[~url_data['Covered by Wildcard']]

# Afficher les résultats
covered_urls.head(), non_covered_urls.head()


(                 URL  Covered by Wildcard
 9   *.infomaniak.com                 True
 11    *.jsdelivr.net                 True,
                                        URL  Covered by Wildcard
 0           addon-stib-mivb.eu.goskope.com                False
 1         gateway-stib-mivb.eu.goskope.com                False
 2  gateway-backup-stib-mivb.eu.goskope.com                False
 3        download-stib-mivb.eu.goskope.com                False
 4        achecker-stib-mivb.eu.goskope.com                False)

In [22]:
import requests

# Fonction pour vérifier l'accessibilité d'une URL et identifier le protocole
def check_url_accessibility(url):
    protocols = ["http://", "https://"]
    for protocol in protocols:
        try:
            response = requests.head(protocol + url, timeout=5, allow_redirects=True)
            if response.status_code == 200:
                return protocol  # Retourne le protocole utilisé si accessible
        except requests.RequestException:
            continue
    return None  # Retourne None si l'URL n'est pas accessible

# Appliquer la fonction à chaque URL non couverte
non_covered_urls['Accessible'] = non_covered_urls['URL'].apply(lambda url: check_url_accessibility(url))

# Ajouter une colonne pour indiquer si l'URL est accessible en HTTP (qui doit être bloqué)
non_covered_urls['HTTP Blocked'] = non_covered_urls['Accessible'].apply(lambda x: x == 'http://')

# Afficher les résultats
non_covered_urls.head()


<ipython-input-22-dbf73339c4fd>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_covered_urls['Accessible'] = non_covered_urls['URL'].apply(lambda url: check_url_accessibility(url))
<ipython-input-22-dbf73339c4fd>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_covered_urls['HTTP Blocked'] = non_covered_urls['Accessible'].apply(lambda x: x == 'http://')


,URL,Covered by Wildcard,Accessible,HTTP Blocked
0,addon-stib-mivb.eu.goskope.com,False,None,False
1,gateway-stib-mivb.eu.goskope.com,False,None,False
2,gateway-backup-stib-mivb.eu.goskope.com,False,None,False
3,download-stib-mivb.eu.goskope.com,False,None,False
4,achecker-stib-mivb.eu.goskope.com,False,None,False


Explications :
Wildcards :

Le code vérifie si une URL correspond à l'un des wildcards en utilisant fnmatch.
Séparation :

Les URLs sont divisées en deux groupes : celles couvertes par un wildcard et celles non couvertes.
Vérification de l'Accessibilité :

Les URLs non couvertes sont ensuite vérifiées pour leur accessibilité via http ou https.

In [27]:
import pandas as pd

# Liste mise à jour des domaines système
os_domains = [
    'microsoft.com', 'windows.net', 'adobe.com', 'office.com', 'skype.com', 'msft.net', 'microsoftonline.com',
    'goskope.com', 'cloudflare.com', 'windows.com', 'azure-automation.com', 'office.net', 'sharepoint.com', 'azure.com'
]

# Liste de départ pour les domaines utilisateur
user_domains = [
    'google.com', 'facebook.com', 'twitter.com', 'linkedin.com', 'youtube.com',
    'vimeo.com', 'wikipedia.org', 'github.com', 'stackoverflow.com',
    'spotify.com', 'netflix.com', 'dropbox.com', 'slack.com', 'zoom.us',
    'whatsapp.com', 'instagram.com', 'pinterest.com', 'amazon.com',
    'ebay.com', 'paypal.com', 'booking.com', 'airbnb.com'
]

# Liste mise à jour des mots-clés pour les URLs système
os_keywords = ['crl', 'auth', 'cdn', 'ocsp', 'ocsp2', 'ocspx', 'cacert', 'cert', 'certs', 'certseid']

# Fonction pour vérifier si une URL est utilisée par l'OS ou par un utilisateur
def classify_url(url):
    if any(domain in url for domain in os_domains):
        return 'OS'
    if any(domain in url for domain in user_domains):
        return 'User'
    if any(keyword in url for keyword in os_keywords):
        return 'OS'
    return 'User'

# Appliquer la fonction de classification
final_data['Category'] = final_data['URL'].apply(classify_url)

# Séparer les URLs système et utilisateur
os_urls = final_data[final_data['Category'] == 'OS']
user_urls = final_data[final_data['Category'] == 'User']

# Sauvegarder les résultats dans des fichiers CSV séparés
os_urls.to_csv('whiteliste-urls-os.csv', index=False)
user_urls.to_csv('whiteliste-urls-user.csv', index=False)

# Télécharger les fichiers générés (si exécuté dans Google Colab)
from google.colab import files
files.download('whiteliste-urls-os.csv')
files.download('whiteliste-urls-user.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Explications :
Liste des Mots-Clés Système :

La liste os_keywords contient tous les mots-clés que vous avez mentionnés : ['crl', 'auth', 'cdn', 'ocsp', 'ocsp2', 'ocspx', 'cacert', 'cert', 'certs', 'certseid'].
Fonction de Classification :

classify_url(url) vérifie si l'URL contient un des domaines de os_domains, un des domaines de user_domains, ou un des mots-clés de os_keywords.
Les URLs qui correspondent aux mots-clés ou aux domaines système sont classées comme OS.
Les autres URLs sont classées comme User.
Séparation et Sauvegarde :

Après la classification, les URLs sont séparées en deux fichiers CSV (whiteliste-urls-os.csv et whiteliste-urls-user.csv).